<a href="https://colab.research.google.com/github/JTStephens18/AudioTranscriptor/blob/main/V3_wave2vec_audioProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install yt-dlp
# Install huggingface audio datasets
! pip install datasets[audio]
! pip install transformers evaluate jiwer
!pip install accelerate -U

In [ ]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor
import subprocess
import os
from pathlib import Path
import torch
import soundfile as sf

In [ ]:
ds = load_dataset('google/MusicCaps', split="train")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list', 'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval'],
    num_rows: 5521
})

In [ ]:
ds = ds.remove_columns(["author_id", "is_balanced_subset", "is_audioset_eval", "audioset_positive_labels"])

In [ ]:
ds = ds.train_test_split(test_size=0.2)

In [ ]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def download_clip(
    video_id,
    output_filename,
    start_time,
    end_time,
    tmp_dir='/musiccaps',
    num_attempts=5,
    url_base='https://www.youtube.com/watch?v='
):

  status = False
  command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_id} --force-keyframes-at-cuts
    """.strip()
  attempts = 0
  while True:
    try:
      # output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
      output = os.system(command)
    except subprocess.CalledProcess.Error as err:
      attempts += 1
      if attempts == num_attempts:
        return status, err.output
    else:
      break

  # Check if video was successfully saved
  status = os.path.exists(output_filename)
  return status, 'Downloaded'


def toUppercase(aspect_list):
  a_list = aspect_list[1:-1]
  new_list = ''
  for word in a_list:
    new_list = ''.join(a_list).upper().replace(',', '')
    new_list = new_list.replace(' ', '|')
  return new_list


def process(example):
  output_path = str(data_dir / f"{example['ytid']}.wav")
  status = True
  aspect_string = toUppercase(example['aspect_list'])
  if not os.path.exists(output_path):
    status = False
    status, log = download_clip(
        example['ytid'],
        output_path,
        example['start_s'],
        example['end_s'],
    )

  example['aspect_string'] = aspect_string
  example["audio"] = output_path
  example['download_status'] = status
  return example

In [ ]:
def process_input_length(example):
  audio = example["audio"]
  example = processor(audio['array'], sampling_rate=audio["sampling_rate"], text=example['aspect_string'])
  # example["input_length"] = len(example["input_values"][0])
  return example

In [ ]:
samples_to_load = 300
cores = 4
sampling_rate = 16000
writer_batch_size = 1000
data_dir = "./music_data"

ds_train = ds["train"].select(range(samples_to_load))
ds_test = ds["test"].select(range(int(samples_to_load*0.2)))

# ds_train = ds["train"]
# ds_test = ds["test"]

data_dir = Path(data_dir)
data_dir.mkdir(exist_ok=True, parents=True)

ds_train = ds_train.map(
    process,
    num_proc=cores,
    writer_batch_size=writer_batch_size,
    keep_in_memory=False
)


ds_train_audio = ds_train.filter(lambda ex: ex["download_status"] == True)
ds_train_audio = ds_train_audio.cast_column('audio', Audio(sampling_rate=sampling_rate))

ds_train_final = ds_train_audio.map(
    process_input_length,
    num_proc=1,
    writer_batch_size=writer_batch_size,
    keep_in_memory=False
)

ds_test = ds_test.map(
    process,
    num_proc=cores,
    writer_batch_size=writer_batch_size,
    keep_in_memory=False
)

ds_test_audio = ds_test.filter(lambda ex: ex["download_status"] == True)
ds_test_audio = ds_test_audio.cast_column('audio', Audio(sampling_rate=sampling_rate))

ds_test = ds_test_audio.map(
    process_input_length,
    num_proc=1,
    writer_batch_size=writer_batch_size,
    keep_in_memory=False
)

Map (num_proc=4):   0%|          | 0/30 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
# audio_sample = ds_train_final[0]["audio"]["array"]
# test_output_path = "output.wav"
# sf.write(test_output_path, audio_sample, 16000)
# ds_train[0]["ytid"]

'ciJOulWFhfA'

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
  processor: AutoProcessor
  padding: Union[bool, str] = 'longest'

  def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
    input_features = [{"input_values": feature["input_values"][0]} for feature in features]
    label_features = [{"input_ids": feature["labels"]} for feature in features]

    batch = self.processor.pad(input_features, padding=self.padding, return_tensors='pt')

    labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors='pt')

    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    batch["labels"] = labels

    return batch

In [ ]:
data_collator =  DataCollatorCTCWithPadding(processor=processor, padding='longest')

In [ ]:
import evaluate

wer_metric = evaluate.load('wer')

In [ ]:
import numpy as np


def compute_metrics(pred):
  pred_logits = pred.predictions
  pred_ids = np.argmax(pred_logits, axis=-1)

  pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

  pred_str = processor.batch_decode(pred_ids)
  label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

  wer = wer_metric.compute(predictions=pred_str, references=label_str)

  return {"wer": wer}

In [ ]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model = AutoModelForCTC.from_pretrained(
    'facebook/wav2vec2-base',
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    ctc_zero_infinity=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_asr_mind_model",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=100,
    eval_steps=100,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    # push_to_hub=True,
)

train_dataset = ds_train_final
test_dataset = ds_test

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


trainer.train()

Step,Training Loss,Validation Loss,Wer
100,7.116800,4.274777,1.000000
200,3.148000,3.786598,1.000000


KeyboardInterrupt: ignored

In [ ]:
for batch in trainer.get_train_dataloader():
  print(batch)
  break

In [ ]:
from transformers import pipeline

audio_file = ds_test[2]["audio"]["array"]
print(audio_file)
# audio_file = audio_file.cast_column("audio", Audio(sampling_rate=16000))
inputs = processor(audio_file, sampling_rate=16000, return_tensors='pt').to("cuda:0")

with torch.no_grad():
  logits = model(**inputs).logits


pred_ids = torch.argmax(logits, dim=1)
trans = processor.batch_decode(pred_ids)
print(trans)

[-0.12915039 -0.12683105  0.01977539 ...  0.22149658  0.24847412
  0.21282959]
t ['<unk>B<unk>']


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
trainer.train_dataset.features["labels"]

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

In [ ]:
decode = processor.decode([3])
decode

'<unk>'